## Import packages

In [1]:
import selenium
from selenium import webdriver

import shutil, time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import Helpers
reload(Helpers)

<module 'Helpers' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\Helpers.py'>

## Configurations

In [2]:
path_to_extension = r'C:\Users\Matth\Desktop\3.12_0'
character_name = "Mercychan"
character_server = "Benediction"
character_region = "US"

## Navigate to character page

In [3]:
char_url, browser = Helpers.load_environment(path_to_extension, character_name, character_server, character_region)

In [4]:
bosses = Helpers.get_phase3_bosses(browser)
bosses

["High Warlord Naj'entus",
 'Supremus',
 'Shade of Akama',
 'Teron Gorefiend',
 'Gurtogg Bloodboil',
 'Reliquary of Souls',
 'Mother Shahraz',
 'The Illidari Council',
 'Illidan Stormrage',
 'Rage Winterchill',
 'Anetheron',
 "Kaz'rogal",
 'Azgalor',
 'Archimonde']

In [86]:
char_info = Helpers.get_character_info(char_url)
char_info

'Mercychan (Benediction US)'

## Main code loop

In [97]:
# Working bosses: Najentus, 
bosses = ["Reliquary of Souls"]
nTanks = 1

In [99]:
def main():
    table_dict = {"Spell" : "-", "Amount" : "-", "Casts" : "-", "Avg Cast" : "-", "Hits" : "-", "Avg Hit" : "-", "Crit %" : "-", "Uptime %" : "-", "Overheal" : "-", "HPS" : "-"}

    player_df = pd.DataFrame(pd.np.empty((0, 5)))
    player_df.columns = ["Boss name", "Character name", "Character server", "HPS", "Rotations"]

    for i, boss in enumerate(bosses):

        Helpers.scroll_click(browser.find_element_by_link_text(boss))
        time.sleep(1)

        # Pick out the top parse of this boss
        search = browser.find_elements_by_class_name("character-table-link")
        Helpers.scroll_click(browser.find_element_by_link_text(search[0].text))
        time.sleep(2)

        player_HPS = Helpers.get_player_HPS(browser, character_name)
        
        if i == 0:
            all_tanks = Helpers.get_tanks(browser) # Returns the 3 tank names
            
        boss_tanks = Helpers.get_boss_tanks(boss, browser, nTanks) 
        
        if boss in ["High Warlord Naj'entus", "Teron Gorefiend", "Mother Shahraz", "Gurtogg Bloodboil", "Reliquary of Souls"]:
            MT = boss_tanks[0]
            
        elif boss in ["Shade of Akama", "Illidan Stormrage"]:
            MT = boss_tanks[1]

        Helpers.scroll_click(browser.find_element_by_link_text(character_name))
        time.sleep(2)
        
        print(f"Boss: {boss}")
        print(f"Main tank: {MT}")
        table_dict = Helpers.get_spell_info(browser, table_dict)
        time.sleep(3)
        
        #----------------------------------

        Helpers.download_cast_sequence(browser)
        time.sleep(3)
        shutil.move("C:/Users/Matth/Downloads/Warcraft Logs - Combat Analysis for Warcraft.csv", "C:/Users/Matth/git/DataAnalysisWorkbooks/warcraftLogs/cast_sequence/casts.csv")

        # Clean the csv
        df = Helpers.clean_csv()

        # Fix regrowth timer
        df = Helpers.fix_regrowth_cast_time(df)

        # Get the rotations
        rotations = Helpers.calculate_rotations(df, MT)
        print(rotations)

        # Save data to csv
        to_append = [boss, character_name, character_server + " " + character_region, player_HPS, rotations]
        series = pd.Series(to_append, index = player_df.columns)
        player_df = player_df.append(series, ignore_index=True)

        # Return to character page
        browser.get(char_url)
        time.sleep(1)

    player_df.to_csv(f"{character_name}.csv", index=None)


if __name__ == "__main__":
    main()

Boss: Reliquary of Souls
Main tank: Cmbreen
          Spell  Amount Casts Avg Cast Hits Avg Hit Crit % Uptime % Overheal  \
0     Lifebloom  311.3k   107     2.9k  618   503.7      -   73.22%   64.64%   
1     Lifebloom     0.0     -        -   11     0.0   9.1%        -  100.00%   
2  Rejuvenation   11.2k     9     1.2k   19   587.8      -   18.70%   50.28%   
3      Regrowth   24.7k    21     1.1k   21    1.1k  71.4%   32.53%   70.54%   
4     Swiftmend   10.9k     4     2.7k    4    2.7k      -        -   44.44%   

     HPS  
0  888.5  
1    0.0  
2   31.9  
3   70.5  
4   31.0  
[['LB 2I', 1.0]]
